# 201710957 컴퓨터과학과 이유진

## 기말고사

## 1) Solidity 코딩

In [72]:
%%writefile  src/Order_Customer.sol
pragma solidity ^0.5.0;


contract Customer{
    
    address payable owner;
    
    bytes2 id; // customer ID  NOT ORDERID
    string name;
    string phone_number;
    address payable addr; // account[0 ] account[1] , account[2] 
    uint mileage = 0;
    bool public isCustomer;

    function addMileage(uint price) public {
        mileage += price /100;
    }
    function MileageUndo(uint price) public {
        mileage -= price/100;
    }
    function setCustomer(address payable addr_, bytes2 _id, string memory  _name, string memory _phone_number) public {
       
        id = _id;
        name = _name;
        phone_number = _phone_number;
        addr = addr_;
        isCustomer = true;
    }
    function getAddress() public view  returns(address payable){
        return addr;
    }
    
    function getTxOriginMsgSender() view public returns(address) {
        return tx.origin;
    }
 
    function getIsCustomer() public view  returns(bool){
        return isCustomer;
    }
}


contract Order{
    address payable owner;
    struct orderitem{
        
        bytes2 orderId;
        string itemName;
        uint quantanty;
        uint orderPrice;
        uint256 orderedTime;
        string status;
        string shipment;
        bool isOrdered;
        
    }
    
    mapping(bytes2 => bool) check_customer;
    mapping(address => Customer) address_customer;
    mapping(bytes2 => Customer) id_customers;

    mapping(address => orderitem) orderlist;
    mapping(bytes2 =>orderitem) id_orderlist;
    
    uint bal;
    uint count;
    
  
    function setCustomerInfo( bytes2 _id, string memory name_, string memory _phone_number, string memory shipmentAddr_) public returns(string memory) {
       
       
        if(check_customer[_id]){
            require(1==0);
        }
         
        check_customer[_id] = true;
        
        Customer customer = new Customer();
        customer.setCustomer(msg.sender, _id,  name_,  _phone_number);
           
        address_customer[msg.sender] = customer;
        id_customers[_id] = customer;
        orderlist[msg.sender] =  orderitem(0x0000,"",0,0,0,"","",false);
        orderlist[msg.sender].shipment = shipmentAddr_;  
        
        return "OK";
     
    }


    function order(bytes2 orderid, string memory item_name, uint quantanty_, uint price) payable public {
       
        if( id_orderlist[orderid].isOrdered ){
            require(1==0);
        }
            
        
        id_customers[orderid] =  address_customer[msg.sender];
        
       
        
        bal += price;
        count++;
        
        orderlist[msg.sender].orderedTime = now;
        orderlist[msg.sender].orderId = orderid;
        orderlist[msg.sender].itemName = item_name;
        orderlist[msg.sender].status = "Ordered";
        orderlist[msg.sender].quantanty = quantanty_;
        orderlist[msg.sender].orderPrice = price;
        orderlist[msg.sender].isOrdered = true;
        
        id_orderlist[orderid] =  orderlist[msg.sender];
        
        
    }
    function printShipment() public view returns (string memory){
        return orderlist[msg.sender].shipment;
    }
    
    function printOrder() public view  returns(bytes2 , string memory, string memory, string  memory){
        orderitem memory temp = orderlist[msg.sender];
        return( temp.orderId , temp.itemName, temp.status, temp.shipment);
    }
    
    function printOrderByID(bytes2 id) public view  returns( bytes2 , string memory, string memory, string  memory){
        orderitem memory temp = id_orderlist[id];
        return ( temp.orderId, temp.itemName, temp.status, temp.shipment);
    }
    function transferTo(address payable temp , uint amount)public{
        temp.transfer(amount);
    }
 
    function refund(bytes2  _orderId) public {
        id_orderlist[_orderId].status = "refunded";
        
        id_customers[_orderId].MileageUndo(id_orderlist[_orderId].orderPrice);
        transferTo(id_customers[_orderId].getAddress() , id_orderlist[_orderId].orderPrice);
        bal -= id_orderlist[_orderId].orderPrice;
        count--;
        id_orderlist[_orderId].orderPrice = 0;
        
        
        
    }
    
    function getBalanceNcount() public view returns(uint, uint, uint ){
        return (count, bal, address(this).balance);
    }
    
    
}

Overwriting src/Order_Customer.sol


In [73]:
!solc src\Order_Customer.sol --combined-json abi,bin > src\Order_Customer.json

In [14]:
%%writefile src\Order_Customer_Deploy.js

var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var fs=require('fs');
var _str = fs.readFileSync("src/Order_Customer.json");
var _json=JSON.parse(_str)

var _abiArray=JSON.parse(_json.contracts["src/Order_Customer.sol:Order"].abi);
var _bin="0x"+_json.contracts["src/Order_Customer.sol:Order"].bin;

   
async function deploy(){
    
    const accounts = await web3.eth.getAccounts();
    console.log("1-1 ) Print address and Balance");
    console.log("address 1: " + accounts[4]);
    const balance1Before = await web3.eth.getBalance(accounts[4]);   
    console.log("->balacne : " + balance1Before);
    
    console.log("address 2: " + accounts[5]);
    const balance2Before = await web3.eth.getBalance(accounts[5]);
    console.log("->balacne: " + balance2Before);

    
    console.log("address 3: " + accounts[6]);
    const balance3Before = await web3.eth.getBalance(accounts[6]);
    console.log("->balacne: " + balance3Before);
    
    const block = await web3.eth.getBlockNumber();
    console.log("\nBlock num : "+ block)
    
    console.log("==============================================================\n");
    
    console.log("1-2 ) Print gas to create Order contract"); 
    
    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function (err, gas){
        if(!err) console.log(">>gas:" + gas +"\n===================================================\n\n");
    });
   
    
    var mulContract = new web3.eth.Contract(_abiArray);
    mulContract
        .deploy({
                data: _bin
        })
        .send({
         from: accounts[0],
         gas: '5121058'
        }, function (error, transactionHash){ 
                console.log("error :" + error);
                console.log("hash : " + transactionHash); 
        })
        .then(function(newContractInstance){
            console.log("address : " + newContractInstance.options.address)
        });
    
 
    
    
}

deploy()

Overwriting src\Order_Customer_Deploy.js


# 답 1-1 1-2

In [15]:
!node  C:\Users\sofod\Code\201710957\src\Order_Customer_Deploy.js

1-1 ) Print address and Balance
address 1: 0x38Ac21e4B9D78a3a4a044e5fdbd3557C85855a43
->balacne : 465000000000000000000
address 2: 0x26dA1a5ADE8FF34E8a7c5e4BAEE288027D15EEC9
->balacne: 625000000000000000000
address 3: 0x0BB84707b013A7cD6b15Dc05589D5674C4E7747a
->balacne: 580002879495000000000

Block num : 804

1-2 ) Print gas to create Order contract
>>gas:2879495


error :null
hash : 0x83a17f48afdc9961aa6a84fee749eeca784b251a8fccaf744a5adf7940d082c9
address : 0xd1dC601594A2e30bd3D358F2aD920C3018c7c56F


### 단계 4: 사용

In [16]:
%%writefile src/Order_Customer_Use.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var fs=require('fs');
var _str = fs.readFileSync("src/Order_Customer.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/Order_Customer.sol:Order"].abi);

var myorder = new web3.eth.Contract(_abiArray, "0xd1dC601594A2e30bd3D358F2aD920C3018c7c56F");

async function doIt() {
    
    const accounts = await web3.eth.getAccounts();
  
    console.log("1-3 ) input Customers Info");
    await myorder.methods.setCustomerInfo("0x0111" , "kim", "010-2017-1111", "111 hongji-dong jongro-gu seoul").send({from: accounts[4],gas:900000});
    await myorder.methods.setCustomerInfo("0x0112" ,"lee", "010-2017-1112", "112 hongji-dong jongro-gu seoul").send({from: accounts[5],gas:900000});
    await myorder.methods.setCustomerInfo("0x0113" ,"lim", "010-2017-1113", "113 hongji-dong jongro-gu seoul").send({from: accounts[6],gas:900000});
    console.log("==============================================================\n");
    
    console.log("1-4 ) Print all of Customers' Shipment info");
    await myorder.methods.printShipment().call({from: accounts[4]}).then(console.log);
    await myorder.methods.printShipment().call({from: accounts[5]}).then(console.log);
    await myorder.methods.printShipment().call({from: accounts[6]}).then(console.log);
    
    console.log("==============================================================\n");
    
   
    console.log("1-5 ) Order");
    
    await myorder.methods.order("0x555", "T-Shirt", 2, 1115).send({from: accounts[4],gas:900000,value :1115});
    await myorder.methods.order("0x556", "T-Shirt", 3, 1116).send({from: accounts[5],gas:900000,value :1116});
    await myorder.methods.order("0x557", "T-Shirt", 4, 1117).send({from: accounts[6],gas:900000,value :1117});
    
    console.log("==============================================================\n");
  
    
    console.log("1-6 ) Print num Of order, sum of order, and balance ");
    console.log("index 0 --> num of order\nindex 1 --> sum of order\nindex 2 --> balacne of contract")
    await myorder.methods.getBalanceNcount().call().then(console.log);  
    console.log("==============================================================\n");
   
    console.log("1-7 ) Print all of  orders");
    
    await myorder.methods.printOrder().call({from: accounts[4]}).then(console.log); 
    await myorder.methods.printOrder().call({from: accounts[5]}).then(console.log); 
    await myorder.methods.printOrder().call({from: accounts[6]}).then(console.log); 
    
    console.log("==============================================================\n");
   
    console.log("1-8 ) Print Order log by 556 ");
    
    await myorder.methods.printOrderByID("0x0556").call({from: accounts[0]}).then(console.log); 
    console.log("==============================================================\n");
   
    console.log("1-9 ) Refund ID 556");
    await myorder.methods.refund("0x0556").send({from: accounts[0]}); 
    console.log("==============================================================\n");
   
    console.log("1-10 ) Print Order log after Refund");
    await myorder.methods.printOrderByID("0x0556").call({from: accounts[0]}).then(console.log); 
    
    console.log("index 0 --> num of order\nindex 1 --> sum of order\nindex 2 --> balacne of contract")
    await myorder.methods.getBalanceNcount().call().then(console.log);  
    
 
    console.log("==============================================================\n");
   
    
}

doIt()


Overwriting src/Order_Customer_Use.js


# 답 1-3 ~1-10

In [17]:
!node src/Order_Customer_Use.js

1-3 ) input Customers Info

1-4 ) Print all of Customers' Shipment info
111 hongji-dong jongro-gu seoul
112 hongji-dong jongro-gu seoul
113 hongji-dong jongro-gu seoul

1-5 ) Order

1-6 ) Print num Of order, sum of order, and balance 
index 0 --> num of order
index 1 --> sum of order
index 2 --> balacne of contract
Result { '0': '3', '1': '3348', '2': '3348' }

1-7 ) Print all of  orders
Result {
  '0': '0x0555',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '111 hongji-dong jongro-gu seoul'
}
Result {
  '0': '0x0556',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '112 hongji-dong jongro-gu seoul'
}
Result {
  '0': '0x0557',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '113 hongji-dong jongro-gu seoul'
}

1-8 ) Print Order log by 556 
Result {
  '0': '0x0556',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '112 hongji-dong jongro-gu seoul'
}

1-9 ) Refund ID 556

1-10 ) Print Order log after Refund
Result {
  '0': '0x0556',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '112 hongji-dong jongro-gu seou